# Redes Generativas Adversarias
Alrededor de la mitad del año 2014 algunos de los investigadores asociados al departamento de informática de la Universidad de Montreal proponen un nuevo marco de referencia para la estimación de un modelo Generativo basado en redes neuronales, el cual presentarón en el artículo por el que posteriormente le dará el nombre a este marco llamado [Generative Adversarial Network](https://arxiv.org/pdf/1406.2661.pdf) *Goodfellow et al* o GAN por sus siglas en inglés como se les conoce popularmente, pero antes que nada se hace necesario responder si quiera brevemente las siguientes cuestiones ¿Qué son los modelos generativos? ¿Qué sosn los procesos adversarios?

De acuerdo con [developers google](https://developers.google.com/) un [modelo generativo](https://developers.google.com/machine-learning/gan/generative) tiene la capacidad de capturar la probabilidad conjunta P(X,Y) de un grupo de datos lo que le permite en algunos casos crear objetos similares a dicho conjunto. Por otro lado, un proceso adversario en su forma mas simple entrena dos modelos, que para el caso específico de las GAN's son llamados `G` y `D`, donde después de un tiempo de optimización **minimax** el entrenamiento tiende a estabilizar haciendo que ambos modelos no aprendan mas el uno del otro *Goodfellow et al* .

El marco de referencia propuesto por *Goodfellow et al* consiste en dos redes neuronales: `G` como se le llama al modelo Generativo y `D` que se refiere a un modelo de clasificación que tratará de identificar que imagnes provienen del conjunto de datos real y cuales vienen de G, a este clsificador se le es llamado Modelo Discriminativo *Goodfellow et al*. Posteriormente de definir la arquitectura de ambos, se opta por entrenar ambos con dos objetivos contrarios: Por parte de `G` se tratará de **maximizar** la probabilidad de que `D` se equivoque. Opuesto, `D` buscará **minimizará** la probabilidad de caer en dicho error. Después de una amplia cantidad de epochs el proceso de aprendizaje se estabilizará. De acuerdo con las estimaciones de los autores el **punto óptimo** de `D` se acercará a un valor constante de $\frac{1}{2}$. Se propone usar modelos de redes neuronales debido a el amplio y fácil uso del método de optimización basado en backpropagation.

## Detalles del entrenamiento
Este proceso inicia con la creación de un vector de ruido denotado por `z` que en algunas ocaciones es llamado *latent vector*. La creación de este vector obedece a una distribución de porbabilidad $p_z(z)$. La cual en la mayoría de los casos es elegida tomada por una distribución uniforme o Normal. Después se representa un mapeo al espacio de datos con $G(z; \theta_g)$ donde la función `G` es diferenciable y una red neuronal cuyos pesos es el conjunto $\theta_g$. En paralelo También se define un función `D` diferenciable en forma de una red neuronal, cuya operación es de la siguiente manera $D(x; \theta_d)$ donde su salida es un escalar que refleja la probabilidad de que x provenga del conjunto de datos mas que de $p_g$. Ya con esta secuencia que también es llamada Fowardpropagation se require los siguientes criterios de optimización: Entrenar `D` para maximizar la probabilidad de que los valores que provengan del dataset los asigne como 1, mientras que los que provengan del la distribución $p_g$ sean asignados como 0. Por otro lado, se entrenará `G` para que `D` asigne $p_g$ como 1. De acuerdo con *Goodfellow et all* la forma matemática de este problema es la siguiente:
$$
min_G~max_D~V(D, G) = \mathbf{E}_{x \sim p_{data}(x)}(logD(x)) + \mathbf{E}_{z \sim p_{z}(z)}(log(1 - D(G(z))))
$$
Para un mejor entendimiento, recorramos la ecuación parte por parte:
* Lo referente a $\mathbf{E}_{x \sim p_{data}(x)}(...)$ tiene varias partes: $\mathbf{E}$ es el valor esperado o promedio y $x \sim p_{data}(x)$ especifíca el conjunto al que se va a evaluar su promedio, que para este caso vienen siendo todos los datos provenientes de los originales.
* Con respecto a $\mathbf{E}_{z \sim p_{z}(z)}(...)$ Las partes funcionan de igual manera que en el anterior, un único distinto es que este apartado $z \sim p_{z}(z)$ especifica que el conjunto obetivo es `z` en este caso.

La figura 1 de *Goodfellow et al* ilustra de gran manera este proceso. A continuación la presentaremos con su respectiva explicación:
![Proceso de entrenamiento de GAN obtenido de Goodfellow et al](../imagenes/figura_1_Goodfell_GAN.PNG)

Explicando el código de colores y líneas: 
1. La línea punteada azul corresponde al valor escalar de `D` en el espacio de datos.
2. La línea de puntos negra corresponde a la distribución que queremos imitar con nuestro modelo generativo
3. La linea continua verde muestra en cada instante de tiempo cual es la distribución generada por `G`
4. La línea horzontal `x` es el espacio de datos completo.
5. Las líneas continuas negras que van de `z` a `x` corresponde el mapeo realizado por $G(z)$ en el espacio de datos.

En el instante de tiempo $(a)$ podemos asumir que las dos distribuciones (las generadas y las originales) son cercanas y `D` es un clasificador acercandose a su punto óptimo.

En el instante de tiempo $(b)$ el discriminador `D` llega a su punto optimo, y por medio del aprendizaje de G este se acerca aún mas a la distribución deseada.

En el instante de tiempo $(c)$ el gradiente porvisto por el discriminador `D` va guiando de mejor manera los pesos de `G` para que este pueda mapear eficazmente el *latent vector* con el espacio de datos donde es mas probable encontrar los datos origunales.

Por último, en el instante de tiempo $(d)$ el discriminador no puede direccionar mas el gradiente de `G` ya que las dos distribuciones son iguales, lo que hace `D` no es capaz de identificar la diferencia por eso su valor a partir de este punto siempre será $\frac{1}{2}$